In [1]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import panel as pn
# Load the data
df = pd.read_excel(r"C:\Users\HP\Desktop\SHG_Booking_Data.xlsx")


In [39]:
# Extract year and month from 'Booking Date'
df['Booking Year'] = df['Booking Date'].dt.year
df['Booking Month'] = df['Booking Date'].dt.strftime('%B')

# Calculate profit
df['Profit/Loss'] = df['Revenue'] + df['Revenue Loss']

# Create widgets for selecting year, month, hotel, and deposit type
year_select = pn.widgets.Select(name="Year", options=['Total'] + sorted(df['Booking Year'].unique()), value='Total', width=150)
month_select = pn.widgets.Select(name="Month", options=['Total'] + sorted(df['Booking Month'].unique()), value='Total', width=150)
hotel_select = pn.widgets.Select(name="Hotel", options=['Total'] + sorted(df['Hotel'].unique()), value='Total', width=150)
deposit_type_select = pn.widgets.Select(name='Deposit Type', options=['Total'] + sorted(df['Deposit Type'].unique()), value='Total', width=150)

# Image at the top
image = pn.pane.PNG(r"C:\Users\HP\Downloads\_aba3d64c-f377-4ebb-8b59-142dde4744ca-removebg-preview.png", width=150, height = 120)

# Create a Markdown pane for "Overview" and "Dashboard"
markdown1 = pn.pane.Markdown("# Overview\n# Dashboard", style={'font-size': '15px', 'text-align': 'center'})

# Widgets
widget = pn.Column(hotel_select, pn.Spacer(height=100), deposit_type_select, pn.Spacer(height=100), year_select, pn.Spacer(height=100 ), month_select)
 
def create_plot(year, month, hotel, deposit_type):
    # Filter the dataframe by the selected year, month, hotel, and deposit type
    filtered_df = df.copy()

    if year != 'Total':
        filtered_df = filtered_df[filtered_df['Booking Year'] == year]
    if month != 'Total':
        filtered_df = filtered_df[filtered_df['Booking Month'] == month]
    if hotel != 'Total':
        filtered_df = filtered_df[filtered_df['Hotel'] == hotel]
    if deposit_type != 'Total':
        filtered_df = filtered_df[filtered_df['Deposit Type'] == deposit_type]    

    # Calculate total revenue, total bookings, and total cancellations
    total_revenue = pn.pane.Markdown(f"<center>Revenue<br><h1>{filtered_df['Revenue'].sum()/1e6:.2f} M</h1></center>", width=150, style={'text-align': 'center', 'border': '2px solid', 'padding': '1px'})
    total_profit = pn.pane.Markdown(f"<center>Profit<br><h1>{filtered_df['Profit'].sum()/1e6:.2f} M</h1></center>", width=150, style={'text-align': 'center', 'border': '2px solid', 'padding': '1px'})
    total_bookings = pn.pane.Markdown(f"<center>Bookings<br><h1>{len(filtered_df)}</h1></center>", width=150, style={'text-align': 'center', 'border': '2px solid', 'padding': '1px'})
    total_cancellations = pn.pane.Markdown(f"<center>Cancellations<br><h1>{filtered_df['Cancelled (0/1)'].sum()}</h1></center>", width=150, style={'text-align': 'center', 'border': '2px solid', 'padding': '1px'})

    # Melt the DataFrame to have 'Revenue' and 'Profit' in the same column
    melted_df = filtered_df.melt(id_vars='Country', value_vars=['Revenue', 'Profit/Loss'], var_name='Type', value_name='Amount')
    # Create a DataFrame with the sum of 'Revenue' and 'Profit' for each country
    grouped_df = melted_df.groupby(['Country', 'Type'])['Amount'].sum().reset_index()
    # Filter the DataFrame to include only the top 10 countries by 'Revenue'
    top_countries = grouped_df[grouped_df['Type'] == 'Revenue'].nlargest(10, 'Amount')['Country']
    # Create your grouped bar plot with title
    revenue_profit_fig = px.bar(grouped_df[grouped_df['Country'].isin(top_countries)].sort_values(by= 'Amount', ascending=True), 
                                y='Country', x='Amount', 
                                color='Type', barmode='group', 
                                title='Revenue and Profit by Country', 
                                color_discrete_map={'Revenue':'lightgreen', 'Profit/Loss':'gray'})
    revenue_profit_fig.update_layout(xaxis_title="", yaxis_title="", legend_title="", title_x=0.5, autosize=False, width=750, height=530, plot_bgcolor='rgba(0,0,0,0)', legend=dict(x=0.8, y=0.05, traceorder="normal"))
    
    filtered_df = filtered_df[filtered_df['Distribution Channel'] != 'Undefined']
    # Create the stacked bar chart with the color map
    dist_fig = px.bar(filtered_df.groupby(["Distribution Channel", "Status"]).size().reset_index(name='Count').sort_values(by= 'Count', ascending=True), 
             x='Distribution Channel', y='Count', 
             color='Status',  
             color_discrete_map={'Check-Out': 'lightgreen', 'Canceled':'gray', 'No-Show':'Salmon'})

    dist_fig.update_layout(xaxis_title="Distribution Channel", yaxis_title="", plot_bgcolor='rgba(0,0,0,0)', barmode='stack', showlegend=False, autosize=False, width=425, height=430)

    cust_fig = px.bar(filtered_df.groupby(["Customer Type", "Status"]).size().reset_index(name='Count').sort_values(by= 'Count', ascending=True), 
             x='Customer Type', y='Count', 
             color='Status',  
             color_discrete_map={'Check-Out': 'lightgreen', 'Canceled':'gray', 'No-Show':'Salmon'})

    cust_fig.update_layout(xaxis_title="Customer Type", yaxis_title="", plot_bgcolor='rgba(0,0,0,0)', barmode='stack', showlegend=False, autosize=False, width=425, height=430)

    status_fig = px.pie(filtered_df.groupby("Status").size().reset_index(name='Booking Count'), 
                        names='Status', color='Status', values='Booking Count', 
                        title='Booking Count(Status)', 
                        color_discrete_map={'Check-Out': 'lightgreen', 'Canceled':'gray', 'No-Show':'Salmon'} )
    status_fig.update_layout(title_x=0.5, plot_bgcolor='rgba(0,0,0,0)', autosize=False, width=870, height=400)

    cancelled_fig = px.line(filtered_df.groupby("Booking Date")["Cancelled (0/1)"].sum().reset_index(), 
                            x='Booking Date', y='Cancelled (0/1)',
                            title='Cancellations Over Time')
    cancelled_fig.update_traces(line=dict(color='gray'))
    cancelled_fig.update_layout(xaxis_title="", yaxis_title="", title_x=0.5, plot_bgcolor='rgba(0,0,0,0)', autosize=False, width=750, height=300)

    # Arrange markdowns and plots in rows
    markdown_row1 = pn.Row(pn.Spacer(width=70), total_revenue, pn.Spacer(width=300), total_profit)
    markdown_row2 = pn.Row(pn.Spacer(width=70), total_bookings, pn.Spacer(width=400), total_cancellations)
    first_row = pn.Column(revenue_profit_fig, cancelled_fig)
    second_row = pn.Column(status_fig, pn.Row(dist_fig, cust_fig))

    return pn.Row(pn.Spacer(width=10), pn.Column(image, markdown1, widget ),
                     pn.Column(pn.Spacer(height=20), markdown_row1, pn.Spacer(height=20), first_row), 
                     pn.Column(pn.Spacer(height=20),markdown_row2, pn.Spacer(height=20), second_row))



# Create a Panel dashboard with filters in a row at the top
dashboard = pn.Column(
    pn.bind(create_plot, year=year_select, month=month_select, hotel=hotel_select, deposit_type=deposit_type_select)
)

# Add CSS to change the background color
css = """
body {
    background: lightblue;
}
"""
pn.extension(raw_css=[css])
# Show the dashboard
dashboard.show()


C:\Users\HP\AppData\Local\Temp\ipykernel_8948\3499730019.py:18: PanelDeprecationWarning:

'style' is deprecated and will be removed in version 1.4, use 'styles' instead.

C:\Users\HP\AppData\Local\Temp\ipykernel_8948\3499730019.py:37: PanelDeprecationWarning:

'style' is deprecated and will be removed in version 1.4, use 'styles' instead.

C:\Users\HP\AppData\Local\Temp\ipykernel_8948\3499730019.py:38: PanelDeprecationWarning:

'style' is deprecated and will be removed in version 1.4, use 'styles' instead.

C:\Users\HP\AppData\Local\Temp\ipykernel_8948\3499730019.py:39: PanelDeprecationWarning:

'style' is deprecated and will be removed in version 1.4, use 'styles' instead.

C:\Users\HP\AppData\Local\Temp\ipykernel_8948\3499730019.py:40: PanelDeprecationWarning:

'style' is deprecated and will be removed in version 1.4, use 'styles' instead.

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:107: FutureWarning:

The behavior of Dateti

Launching server at http://localhost:54616
